只需要执行start training之前代码，然后执行EGG不同组合代码

#### load data

In [ ]:
import transformers
import torch
from transformers import BertModel, BertTokenizerFast

In [ ]:
weight = 'bert-base-uncased'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
max_len = 35

In [ ]:
import json


train_path = '/content/drive/MyDrive/ner/datas/Election-Trec/train.json'
test_path = '/content/drive/MyDrive/ner/datas/Election-Trec/test.json'
train_file = json.load(open(train_path,'r',encoding='utf-8'))
test_file = json.load(open(test_path, 'r', encoding='utf-8'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Append all words, eye-tracking signals, EEG signals and tags from training json to list
train_sens, train_tags = [],[]
train_Feature = []
train_word_nums = []

sens = ''
nums = 0
for key in train_file.keys():
    tags = []
    features = []
    items = train_file[key]
    sens = ''
    nums = 0
    for item in items:
        sens += item[0]
        sens += ' '
        features.append(item[1:-1])               # ET+EEG: [1: -1]
        tags.append(item[-1])
        nums += 1
    train_sens.append(sens.strip())
    train_word_nums.append(nums)
    train_Feature.append(features)
    train_tags.append(tags)

In [ ]:
# Append all words, eye-tracking signals, EEG signals and tags from testing json to list
test_sens, test_tags = [],[]
test_Feature = []
test_word_nums = []

sens = ''
nums = 0
for key in test_file.keys():
    tags = []
    features = []
    items = test_file[key]
    sens = ''
    nums = 0
    for item in items:
        sens += item[0]
        sens += ' '
        features.append(item[1:-1])                # ET+EEG: [1: -1]
        tags.append(item[-1])
        nums += 1
    test_sens.append(sens.strip())
    test_word_nums.append(nums)
    test_Feature.append(features)
    test_tags.append(tags)

In [ ]:
len(test_sens)

3027

#### build dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(weight)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
label_to_ids = {'none': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4, "O": 5}
# label_to_ids = {'O': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4}

In [ ]:
from tqdm import tqdm
import numpy as np

class MyDataset(Dataset):
    def __init__(self, texts, old_features, tags):
        self.texts = texts
        self.tags = tags
        self.old_features = old_features

        self.labels = []
        self.tokens = []
        self.features = []

        self.input_ids = None
        self.attention_masks = None

    def encode(self):
        for i in tqdm(range(len(self.texts))):
          text = self.texts[i]
          tag = self.tags[i]
          feature = self.old_features[i]
          tags, tokens, features = align_label(text, tag, feature)
          self.labels.append(tags)
          self.tokens.append(tokens)
          self.features.append(features)

        self.features = np.array(self.features,float)
        self.inputs = tokenizer(self.texts, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
        self.input_ids = self.inputs['input_ids']
        self.attention_masks = self.inputs['attention_mask']

    def __getitem__(self, idx):
        return self.input_ids[idx,:], self.attention_masks[idx,:], self.tokens[idx], torch.tensor(self.features[idx],dtype=torch.float32), torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.input_ids)

In [ ]:
label_all_tokens = True



def align_label(text, labels, features):
    input = tokenizer(text, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
    word_ids = input.word_ids()
    input_ids = input['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    previous_word_idx = None
    new_labels, new_features = [], []
    no_features = [0 for _ in range(1, 26)]

    for word_idx in word_ids:
        if word_idx is None:
            new_labels.append('none')
            new_features.append(no_features)
        elif word_idx != previous_word_idx:
            try:
                new_labels.append(labels[word_idx])
                new_features.append(features[word_idx])
            except:
                new_labels.append('none')
                new_features.append(no_features)
        else:
            try:
                new_labels.append(labels[word_idx] if label_all_tokens else 'none')
                new_features.append(features[word_idx] if label_all_tokens else no_features)
            except:
                new_labels.append('none')
                new_features.append(no_features)
        previous_word_idx = word_idx

    label_ids = [label_to_ids[label] for label in new_labels]
    return label_ids, tokens, new_features


In [ ]:
train_dataset = MyDataset(train_sens, train_Feature, train_tags)
train_dataset.encode()

100%|██████████| 24210/24210 [00:08<00:00, 2808.83it/s]


In [ ]:
test_dataset = MyDataset(test_sens, test_Feature, test_tags)
test_dataset.encode()

100%|██████████| 3027/3027 [00:01<00:00, 2886.57it/s]


In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=128)

#### construct bert  model

In [ ]:
import torch.nn as nn
import torch



import torch.nn as nn
import torch

class BertNerModel(nn.Module):
    def __init__(self,num_labels):
        super(BertNerModel,self).__init__()

        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768+8,num_labels)

    def forward(self,input_ids,attention_mask,extra_features,token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        pooled_output = outputs[0]
        bert_outputs = self.dropout(pooled_output)
        EGG = extra_features[:, :, -8:]
        outputs = torch.concat((bert_outputs,EGG),-1)
        # print(bert_outputs.shape)
        # print(EGG.shape)
        # print(outputs.shape)

        # outputs = bert_outputs
        outputs = self.classifier(outputs)

        return outputs


#### evaluate

In [ ]:
def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
      kw_list = []
      nkw_list = ""
      for j in range(len(raw_tags[i])):
          item = raw_tags[i][j]
          if item == 0:
              continue
          if poss !=None and j in poss[i]:
              continue
          # if item == 5:
          #     continue
          if item == 4:
              kw_list.append(str(words_set[j][i]))
          if item == 1:
              nkw_list += str(words_set[j][i])
          if item == 2:
              nkw_list += " "
              nkw_list += str(words_set[j][i])
          if item == 3:
              nkw_list += " "
              nkw_list += str(words_set[j][i])
              kw_list.append(nkw_list)
              nkw_list = ""

      true_tags.append(kw_list)
    return true_tags

In [ ]:
def evaluate(predict_data, target_data, topk=3):
  TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
  for index, words in enumerate(predict_data):
      y_pred, y_true = None, target_data[index]

      if type(predict_data) == str:
          words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
          y_pred = [i[0] for i in words]
      elif type(predict_data) == list:
          y_pred = words

      y_pred = y_pred[0: topk]
      TRUE_NUM = len(set(y_pred) & set(y_true))
      TRUE_COUNT += TRUE_NUM
      PRED_COUNT += len(y_pred)
      GOLD_COUNT += len(y_true)
  # compute P
  if PRED_COUNT != 0:
      p = (TRUE_COUNT / PRED_COUNT)
  else:
      p = 0
  # compute R
  if GOLD_COUNT != 0:
      r = (TRUE_COUNT / GOLD_COUNT)
  else:
      r = 0
  # compute F1
  if (r + p) != 0:
      f1 = ((2 * r * p) / (r + p))
  else:
      f1 = 0

  p = round(p * 100, 2)
  r = round(r * 100, 2)
  f1 = round(f1 * 100, 2)

  return p, r, f1

In [ ]:
import numpy as np

def calculate_f1(y_pred, y_true):
    # flatten and convert to numpy array
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    mask = np.where(y_true != 0)

    y_true = y_true[mask]
    y_pred = y_pred[mask]

    return y_pred, y_true

#### start training

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, AdamW

model = BertNerModel(num_labels=6)
model = model.to(device)

optim = AdamW(model.parameters(),lr=5e-5,weight_decay=1e-2)
loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score

epochs = 5
best_f1 = 0.0
for epoch in tqdm(range(epochs)):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i,batch in enumerate(train_dataloader):
        optim.zero_grad()
        input_ids, attention_masks, _, features, tags = batch
        pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device))

        loss = loss_fn(pred_tags.permute(0,2,1),tags.to(device))
        loss = loss.mean()
        loss.backward()
        optim.step()

        pred_tags = F.softmax(pred_tags,dim=-1)
        pred_tags = torch.argmax(pred_tags,dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [],[]
    for i,batch in enumerate(test_dataloader):
      input_ids, attention_masks, tokens, features, tags = batch
      with torch.no_grad():
          for module in model.modules():
              if isinstance(module, nn.Dropout):
                  module.p = 0
                  module.train(False)
          pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device))
          pred_tags = F.softmax(pred_tags,dim=-1)
          pred_tags = torch.argmax(pred_tags,dim=-1)

      y_pred, y_true = calculate_f1(pred_tags, tags)
      label_true.extend(y_true)
      label_pred.extend(y_pred)

      # more balance evaluate
      poss = []
      for i in range(len(tags)):
          pos = []
          for j in range(len(tags[i])):
              if tags[i][j] == 0:
                  pos.append(j)
          poss.append(pos)

      kw_true.extend(TagConvert(tags,tokens))
      kw_pred.extend(TagConvert(pred_tags,tokens,poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(),'./pretrain_pt/bert_EGG.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch+1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))

 20%|██        | 1/5 [38:06<2:32:27, 2286.83s/it]

epoch1:  loss:0.20   train_f1_value:0.26  test_f1_value:0.53  kw_f1_value:59.86


 40%|████      | 2/5 [1:16:06<1:54:14, 2284.76s/it]

epoch2:  loss:0.10   train_f1_value:0.63  test_f1_value:0.67  kw_f1_value:68.35


 60%|██████    | 3/5 [1:53:44<1:15:53, 2276.63s/it]

epoch3:  loss:0.05   train_f1_value:0.81  test_f1_value:0.69  kw_f1_value:71.90


 80%|████████  | 4/5 [2:31:34<37:54, 2274.67s/it]  

epoch4:  loss:0.03   train_f1_value:0.89  test_f1_value:0.69  kw_f1_value:71.31


100%|██████████| 5/5 [3:09:32<00:00, 2274.52s/it]

epoch5:  loss:0.02   train_f1_value:0.93  test_f1_value:0.70  kw_f1_value:71.54


#### inference

In [ ]:
model = BertNerModel(num_labels=6)
model.load_state_dict(torch.load('./pretrain_pt/bert_EGG.pt'))
model = model.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score

model.eval()
kw_true, kw_pred = [], []
label_true, label_pred = [],[]
for i,batch in enumerate(test_dataloader):
    input_ids, attention_masks, tokens, extra_features, tags = batch
    with torch.no_grad():
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.p = 0
                module.train(False)
        #pred_tags = model(input_ids.to(device), attention_masks.to(device))
        pred_tags = model(input_ids.to(device), attention_masks.to(device), extra_features.to(device))
        pred_tags = F.softmax(pred_tags,dim=-1)
        pred_tags = torch.argmax(pred_tags,dim=-1)

    y_pred, y_true = calculate_f1(pred_tags, tags)
    label_true.extend(y_true)
    label_pred.extend(y_pred)

    # more balance evaluate
    poss = []
    for i in range(len(tags)):
        pos = []
        for j in range(len(tags[i])):
            if tags[i][j] == 0:
                pos.append(j)
        poss.append(pos)

    kw_true.extend(TagConvert(tags,tokens))
    kw_pred.extend(TagConvert(pred_tags,tokens,poss))

label_f1 = f1_score(label_true, label_pred, average='macro')
P, R, F1 = evaluate(kw_true, kw_pred)

In [ ]:
print(P)
print(R)
print(F1)

70.43
73.43
71.9


In [ ]:
#######################################BERT+EEG组合

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score
import torch.nn as nn
import torch

# 定义模型类
class BertNerModel(nn.Module):
    def __init__(self, num_labels, eeg_combo_dim):
        super(BertNerModel, self).__init__()
        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + eeg_combo_dim, num_labels)

    def forward(self, input_ids, attention_mask, extra_features, eeg_combo):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        bert_outputs = self.dropout(outputs[0])

        # 动态选择 EEG 特征
        if eeg_combo == 'EEG1':
            eeg_features = extra_features[:, :, -8:-7]
        elif eeg_combo == 'EEG2':
            eeg_features = extra_features[:, :, -7:-6]
        elif eeg_combo == 'EEG3':
            eeg_features = extra_features[:, :, -6:-5]
        elif eeg_combo == 'EEG4':
            eeg_features = extra_features[:, :, -5:-4]
        elif eeg_combo == 'EEG5':
            eeg_features = extra_features[:, :, -4:-3]
        elif eeg_combo == 'EEG6':
            eeg_features = extra_features[:, :, -3:-2]
        elif eeg_combo == 'EEG7':
            eeg_features = extra_features[:, :, -2:-1]
        elif eeg_combo == 'EEG8':
            eeg_features = extra_features[:, :, -1:]
        elif eeg_combo == 'EEG1.2':
            eeg_features = extra_features[:, :, -8:-6]
        elif eeg_combo == 'EEG2.3':
            eeg_features = extra_features[:, :, -7:-5]
        elif eeg_combo == 'EEG3.4':
            eeg_features = extra_features[:, :, -6:-4]
        elif eeg_combo == 'EEG4.5':
            eeg_features = extra_features[:, :, -5:-3]
        elif eeg_combo == 'EEG5.6':
            eeg_features = extra_features[:, :, -4:-2]
        elif eeg_combo == 'EEG6.7':
            eeg_features = extra_features[:, :, -3:-1]
        elif eeg_combo == 'EEG7.8':
            eeg_features = extra_features[:, :, -2:]

        outputs = torch.concat((bert_outputs, eeg_features), -1)
        outputs = self.classifier(outputs)
        return outputs

# 训练和评估函数
def train_and_evaluate(train_dataloader, test_dataloader, eeg_combo, num_labels=6, epochs=5, lr=5e-5):
    eeg_combo_dim = len(eeg_combo.split('.'))  # EEG 组合维度

    model = BertNerModel(num_labels=num_labels, eeg_combo_dim=eeg_combo_dim)
    model = model.to(device)

    optim = AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0).to(device)

    best_f1 = 0.0

    # Training loop
    for epoch in tqdm(range(epochs)):
        loss_value = 0.0
        model.train()
        label_true, label_pred = [], []
        for i, batch in enumerate(train_dataloader):
            optim.zero_grad()
            input_ids, attention_masks, _, features, tags = batch
            pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device), eeg_combo)

            loss = loss_fn(pred_tags.permute(0, 2, 1), tags.to(device))
            loss = loss.mean()
            loss.backward()
            optim.step()

            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            loss_value += loss.item()

        label_train_f1 = f1_score(label_true, label_pred, average='macro')

        # Evaluation loop
        model.eval()
        kw_true, kw_pred = [], []
        label_true, label_pred = [], []
        for i, batch in enumerate(test_dataloader):
            input_ids, attention_masks, tokens, features, tags = batch
            with torch.no_grad():
                for module in model.modules():
                    if isinstance(module, nn.Dropout):
                        module.p = 0
                        module.train(False)
                pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device), eeg_combo)
                pred_tags = F.softmax(pred_tags, dim=-1)
                pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)

            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

        label_f1 = f1_score(label_true, label_pred, average='macro')
        P, R, F1 = evaluate(kw_true, kw_pred)

        # Save the best model for the given EEG combination
        if F1 > best_f1:
            best_f1 = F1
            torch.save(model.state_dict(), f'/content/drive/MyDrive/ner/pretrain_pt/bert_{eeg_combo}.pt')

        print(f"epoch {epoch+1}: loss: {loss_value / len(train_dataloader):.2f} train_f1: {label_train_f1:.2f} test_f1: {label_f1:.2f} kw_f1: {F1:.2f}")

    return P, R, F1

# 使用不同的 EEG 组合进行训练和评估
eeg_combos = ['EEG1', 'EEG2', 'EEG3', 'EEG4', 'EEG5', 'EEG6', 'EEG7', 'EEG8',
              'EEG1.2', 'EEG2.3', 'EEG3.4', 'EEG4.5', 'EEG5.6', 'EEG6.7', 'EEG7.8']

for eeg_combo in eeg_combos:
    print(f"Training with EEG combo: {eeg_combo}")
    P, R, F1 = train_and_evaluate(train_dataloader, test_dataloader, eeg_combo)
    print(f"EEG combo: {eeg_combo} - Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")


Training with EEG combo: EEG1


 20%|██        | 1/5 [01:20<05:21, 80.45s/it]

epoch 1: loss: 0.21 train_f1: 0.25 test_f1: 0.45 kw_f1: 56.86


 40%|████      | 2/5 [02:40<04:01, 80.43s/it]

epoch 2: loss: 0.11 train_f1: 0.60 test_f1: 0.63 kw_f1: 67.64


...

epoch 3: loss: 0.05 train_f1: 0.83 test_f1: 0.70 kw_f1: 71.15


 80%|████████  | 4/5 [05:19<01:19, 79.81s/it]

epoch 4: loss: 0.03 train_f1: 0.90 test_f1: 0.71 kw_f1: 71.91


100%|██████████| 5/5 [06:39<00:00, 79.88s/it]

epoch 5: loss: 0.02 train_f1: 0.93 test_f1: 0.73 kw_f1: 72.41
EEG combo: EEG7.8 - Precision: 74.49, Recall: 70.43, F1 Score: 72.41


In [ ]:
##############################################

In [ ]:
fs_num = 25  # 定义额外特征的数量

In [ ]:
#####################

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.att_weight = nn.Parameter(torch.Tensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.att_weight)
        self.tanh = nn.Tanh()

    def attention_layer(self, h, mask):
        att_weight = self.att_weight.expand(mask.shape[0], -1, -1)  # B*H*1
        att_score = torch.bmm(self.tanh(h), att_weight)  # B*L*H  *  B*H*1 -> B*L*1

        # mask, remove the effect of 'PAD'
        mask = mask.unsqueeze(dim=-1)  # B*L*1
        att_score = att_score.masked_fill(mask.eq(0), float('-inf'))  # B*L*1
        att_weight = F.softmax(att_score, dim=1)  # B*L*1

        reps = h * att_weight  # B*L*H *  B*L*1 -> B*L*H
        reps = self.tanh(reps)  # B*L*H
        return reps, att_weight


class BertNerModelWithAttention(nn.Module):
    def __init__(self, num_labels):
        super(BertNerModelWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.layernorm = nn.LayerNorm(normalized_shape=768)  # 使用BERT的输出维度进行LayerNorm
        self.relu = nn.ReLU()
        self.linear_dropout = nn.Dropout(0.1)
        self.attention = Attention(768)
        self.classifier = nn.Linear(768 + 8, num_labels)  # 修改为18维

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]  # [batch_size, seq_len, hidden_dim]
        sequence_output = self.dropout(sequence_output)

        # 添加LayerNorm、ReLU和Dropout操作
        normalized_output = self.layernorm(sequence_output)
        activated_output = self.relu(normalized_output)
        dropout_output = self.linear_dropout(activated_output)

        context_vector, attention_weights = self.attention.attention_layer(dropout_output, attention_mask)  # [batch_size, seq_len, hidden_dim]

        # 只使用前18个ET特征
        et_features = extra_features[:, :, -8:]  # 提取前18维度的特征

        # 直接拼接 context_vector 和 et_features
        combined_output = torch.cat((context_vector, et_features), dim=-1)  # [batch_size, seq_len, hidden_dim + 18]
        logits = self.classifier(combined_output)  # [batch_size, seq_len, num_labels]

        return logits, attention_weights


In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from sklearn.metrics import f1_score

# model = BertNerModelWithSoftAttention(num_labels=6)
# model = model.to(device)


model = BertNerModelWithAttention(num_labels=6)
model = model.to(device)


#optim = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
optim = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-2)  # 尝试降低学习率

loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

epochs = 5
best_f1 = 0.0
num_labels = 6

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
# 训练模型
import torch.nn.functional as F

epochs = 5
best_f1 = 0.0
for epoch in tqdm(range(epochs)):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i, batch in enumerate(train_dataloader):
        optim.zero_grad()
        input_ids, attention_masks, _, features, tags = batch
        pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device))

        loss = loss_fn(pred_tags.permute(0, 2, 1), tags.to(device))
        loss = loss.mean()
        loss.backward()
        optim.step()

        pred_tags = F.softmax(pred_tags, dim=-1)
        pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [], []
    for i, batch in enumerate(test_dataloader):
        input_ids, attention_masks, tokens, features, tags = batch
        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
                    module.train(False)
            pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device))
            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        poss = []
        for i in range(len(tags)):
            pos = []
            for j in range(len(tags[i])):
                if tags[i][j] == 0:
                    pos.append(j)
            poss.append(pos)

        kw_true.extend(TagConvert(tags, tokens))
        kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(), './pretrain_pt/bert_with_soft_EGG.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch + 1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))


  0%|          | 0/5 [00:00<?, ?it/s]F:\anaconda\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 20%|██        | 1/5 [01:59<07:57, 119.42s/it]

epoch1:  loss:0.98   train_f1_value:0.16  test_f1_value:0.16  kw_f1_value:1.50


 40%|████      | 2/5 [03:57<05:56, 119.00s/it]

epoch2:  loss:0.89   train_f1_value:0.16  test_f1_value:0.19  kw_f1_value:1.37


 60%|██████    | 3/5 [05:55<03:57, 118.84s/it]

epoch3:  loss:0.85   train_f1_value:0.16  test_f1_value:0.19  kw_f1_value:0.00


 80%|████████  | 4/5 [07:54<01:58, 118.72s/it]

epoch4:  loss:0.81   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.00


100%|██████████| 5/5 [09:50<00:00, 118.04s/it]

epoch5:  loss:0.78   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.00


In [ ]:
model = BertNerModelWithAttention(num_labels=6)
model.load_state_dict(torch.load('./pretrain_pt/bert_with_soft_EGG.pt'))
model = model.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
import torch.nn as nn

# 加载最佳模型权重
model.load_state_dict(torch.load('./pretrain_pt/bert_with_soft_EGG.pt'))
model.eval()


def inference_and_evaluate(test_dataloader, model, device):
    kw_true, kw_pred = [], []
    label_true, label_pred = [], []

    for i, batch in enumerate(test_dataloader):
        input_ids, attention_masks, tokens, features, tags = batch
        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
                    module.train(False)
            outputs = model(input_ids.to(device), attention_masks.to(device), features.to(device))
            pred_tags = outputs[0] if isinstance(outputs, tuple) else outputs  # Handle tuple output
            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        # more balance evaluate
        poss = []
        for i in range(len(tags)):
            pos = []
            for j in range(len(tags[i])):
                if tags[i][j] == 0:
                    pos.append(j)
            poss.append(pos)

        kw_true.extend(TagConvert(tags, tokens))
        kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    return label_f1, P, R, F1

# 调用推理和评价函数
label_f1, P, R, F1 = inference_and_evaluate(test_dataloader, model, device)

print(f"Label F1 Score: {label_f1:.2f}")
print(f"Precision: {P:.2f}")
print(f"Recall: {R:.2f}")
print(f"F1 Score: {F1:.2f}")


Label F1 Score: 0.16
Precision: 1.37
Recall: 1.66
F1 Score: 1.50


In [ ]:
#################################soft——不同EGG组合

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score
import torch.nn as nn
import torch

# Attention 和 BertNerModelWithAttention 定义
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.att_weight = nn.Parameter(torch.Tensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.att_weight)
        self.tanh = nn.Tanh()

    def attention_layer(self, h, mask):
        att_weight = self.att_weight.expand(mask.shape[0], -1, -1)  # B*H*1
        att_score = torch.bmm(self.tanh(h), att_weight)  # B*L*H  *  B*H*1 -> B*L*1

        # mask, remove the effect of 'PAD'
        mask = mask.unsqueeze(dim=-1)  # B*L*1
        att_score = att_score.masked_fill(mask.eq(0), float('-inf'))  # B*L*1
        att_weight = F.softmax(att_score, dim=1)  # B*L*1

        reps = h * att_weight  # B*L*H *  B*L*1 -> B*L*H
        reps = self.tanh(reps)  # B*L*H
        return reps, att_weight


class BertNerModelWithAttention(nn.Module):
    def __init__(self, num_labels, eeg_combo_dim):
        super(BertNerModelWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.layernorm = nn.LayerNorm(normalized_shape=768)  # 使用BERT的输出维度进行LayerNorm
        self.relu = nn.ReLU()
        self.linear_dropout = nn.Dropout(0.1)
        self.attention = Attention(768)
        self.classifier = nn.Linear(768 + eeg_combo_dim, num_labels)

    def forward(self, input_ids, attention_mask, extra_features, eeg_combo):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]  # [batch_size, seq_len, hidden_dim]
        sequence_output = self.dropout(sequence_output)

        # 添加LayerNorm、ReLU和Dropout操作
        normalized_output = self.layernorm(sequence_output)
        activated_output = self.relu(normalized_output)
        dropout_output = self.linear_dropout(activated_output)

        context_vector, attention_weights = self.attention.attention_layer(dropout_output, attention_mask)  # [batch_size, seq_len, hidden_dim]

        # 动态选择 EEG 特征
        if eeg_combo == 'EEG1':
            eeg_features = extra_features[:, :, -8:-7]
        elif eeg_combo == 'EEG2':
            eeg_features = extra_features[:, :, -7:-6]
        elif eeg_combo == 'EEG3':
            eeg_features = extra_features[:, :, -6:-5]
        elif eeg_combo == 'EEG4':
            eeg_features = extra_features[:, :, -5:-4]
        elif eeg_combo == 'EEG5':
            eeg_features = extra_features[:, :, -4:-3]
        elif eeg_combo == 'EEG6':
            eeg_features = extra_features[:, :, -3:-2]
        elif eeg_combo == 'EEG7':
            eeg_features = extra_features[:, :, -2:-1]
        elif eeg_combo == 'EEG8':
            eeg_features = extra_features[:, :, -1:]
        elif eeg_combo == 'EEG1.2':
            eeg_features = extra_features[:, :, -8:-6]
        elif eeg_combo == 'EEG2.3':
            eeg_features = extra_features[:, :, -7:-5]
        elif eeg_combo == 'EEG3.4':
            eeg_features = extra_features[:, :, -6:-4]
        elif eeg_combo == 'EEG4.5':
            eeg_features = extra_features[:, :, -5:-3]
        elif eeg_combo == 'EEG5.6':
            eeg_features = extra_features[:, :, -4:-2]
        elif eeg_combo == 'EEG6.7':
            eeg_features = extra_features[:, :, -3:-1]
        elif eeg_combo == 'EEG7.8':
            eeg_features = extra_features[:, :, -2:]

        combined_output = torch.cat((context_vector, eeg_features), dim=-1)  # [batch_size, seq_len, hidden_dim + eeg_combo_dim]
        logits = self.classifier(combined_output)  # [batch_size, seq_len, num_labels]

        return logits, attention_weights

# 训练和评估函数
def train_and_evaluate(train_dataloader, test_dataloader, eeg_combo, num_labels=6, epochs=5, lr=5e-5):
    eeg_combo_dim = len(eeg_combo.split('.'))  # EEG 组合维度

    model = BertNerModelWithAttention(num_labels=num_labels, eeg_combo_dim=eeg_combo_dim)
    model = model.to(device)

    optim = AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0).to(device)

    best_f1 = 0.0

    # Training loop
    for epoch in tqdm(range(epochs)):
        loss_value = 0.0
        model.train()
        label_true, label_pred = [], []
        for i, batch in enumerate(train_dataloader):
            optim.zero_grad()
            input_ids, attention_masks, _, features, tags = batch
            pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device), eeg_combo)

            loss = loss_fn(pred_tags.permute(0, 2, 1), tags.to(device))
            loss = loss.mean()
            loss.backward()
            optim.step()

            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            loss_value += loss.item()

        label_train_f1 = f1_score(label_true, label_pred, average='macro')

        # Evaluation loop
        model.eval()
        kw_true, kw_pred = [], []
        label_true, label_pred = [], []
        for i, batch in enumerate(test_dataloader):
            input_ids, attention_masks, tokens, features, tags = batch
            with torch.no_grad():
                for module in model.modules():
                    if isinstance(module, nn.Dropout):
                        module.p = 0
                        module.train(False)
                pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device), eeg_combo)
                pred_tags = F.softmax(pred_tags, dim=-1)
                pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)

            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

        label_f1 = f1_score(label_true, label_pred, average='macro')
        P, R, F1 = evaluate(kw_true, kw_pred)

        # Save the best model for the given EEG combination
        if F1 > best_f1:
            best_f1 = F1
            torch.save(model.state_dict(), f'/content/drive/MyDrive/ner/pretrain_pt/bert_soft_{eeg_combo}.pt')
        print(f"epoch {epoch+1}: loss: {loss_value / len(train_dataloader):.2f} train_f1: {label_train_f1:.2f} test_f1: {label_f1:.2f} kw_f1: {F1:.2f}")

    return P, R, F1

# 使用不同的 EEG 组合进行训练和评估
eeg_combos = ['EEG1', 'EEG2', 'EEG3', 'EEG4', 'EEG5', 'EEG6', 'EEG7', 'EEG8',
              'EEG1.2', 'EEG2.3', 'EEG3.4', 'EEG4.5', 'EEG5.6', 'EEG6.7', 'EEG7.8']

for eeg_combo in eeg_combos:
    print(f"Training with EEG combo: {eeg_combo}")
    P, R, F1 = train_and_evaluate(train_dataloader, test_dataloader, eeg_combo)
    print(f"EEG combo: {eeg_combo} - Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")


Training with EEG combo: EEG1


model.safetensors:  83%|########3 | 367M/440M [00:00<?, ?B/s]


 20%|██        | 1/5 [01:20<05:21, 80.26s/it]

epoch 1: loss: 0.92 train_f1: 0.16 test_f1: 0.20 kw_f1: 0.00



 40%|████      | 2/5 [02:39<03:58, 79.50s/it]

...

epoch 3: loss: 0.61 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00



 80%|████████  | 4/5 [05:16<01:19, 79.13s/it]

epoch 4: loss: 0.51 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00



100%|██████████| 5/5 [06:35<00:00, 79.17s/it]

epoch 5: loss: 0.43 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00
EEG combo: EEG7.8 - Precision: 0.00, Recall: 0.00, F1 Score: 0.00


 ###定义词级别和句子级别的注意力层

层注意力 2.0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from sklearn.metrics import f1_score
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import BertModel
from torch import nn
from torch.cuda.amp import autocast, GradScaler

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.context = nn.Parameter(torch.FloatTensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.context)

    def forward(self, x, mask=None):
        attention_in = torch.tanh(torch.matmul(x, self.context))
        attention_in = torch.squeeze(attention_in, -1)
        if mask is not None:
            attention_in = attention_in * mask.float()
        attention_weights = F.softmax(attention_in, dim=-1)
        weighted_sum = torch.bmm(attention_weights.unsqueeze(1), x).squeeze(1)
        return weighted_sum

class BertHANModel(nn.Module):
    def __init__(self, num_labels, hidden_dim=768, rnn_dim=256):
        super(BertHANModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.word_attention = Attention(hidden_dim)
        self.rnn = nn.GRU(hidden_dim, rnn_dim, batch_first=True, bidirectional=True)
        self.sentence_attention = Attention(rnn_dim * 2)
        self.classifier = nn.Linear(rnn_dim * 2 + 8, num_labels)  # 修改为17维

    def forward(self, input_ids, attention_mask, extra_features):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs[0]  # shape: (batch_size, seq_length, hidden_dim)

        # Word-level attention
        word_attention_output = self.word_attention(sequence_output)

        # Sentence-level GRU
        rnn_output, _ = self.rnn(word_attention_output.unsqueeze(1))

        # Sentence-level attention
        sentence_attention_output = self.sentence_attention(rnn_output)

        # 确保 sentence_attention_output 是三维张量
        if sentence_attention_output.dim() == 2:
            # 将其调整为三维张量 [batch_size, seq_len, rnn_dim * 2]
            sentence_attention_output = sentence_attention_output.unsqueeze(1).expand(-1, extra_features.size(1), -1)

        # 只使用前 17 个 ET 特征
        eye_tracking_features = extra_features[:, :, -8:]  # [batch_size, seq_len, 17]

        # 打印维度以调试
        # print(f"Adjusted sentence_attention_output shape: {sentence_attention_output.shape}")
        # print(f"eye_tracking_features shape: {eye_tracking_features.shape}")

        # 拼接特征
        combined_output = torch.cat((sentence_attention_output, eye_tracking_features), dim=-1)  # [batch_size, seq_len, rnn_dim * 2 + 17]

        logits = self.classifier(combined_output)  # shape: (batch_size, seq_len, num_labels)

        return logits  # 返回 shape: (batch_size, seq_length, num_labels)





def calculate_f1(y_pred, y_true):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    mask = np.where(y_true != 0)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    return y_pred, y_true

def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
        kw_list = []
        nkw_list = ""
        for j in range(len(raw_tags[i])):
            item = raw_tags[i][j]
            if item == 0:
                continue
            if poss != None and j in poss[i]:
                continue
            if item == 4:
                kw_list.append(str(words_set[j][i]))
            if item == 1:
                nkw_list += str(words_set[j][i])
            if item == 2:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
            if item == 3:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
                kw_list.append(nkw_list)
                nkw_list = ""
        true_tags.append(kw_list)
    return true_tags

def evaluate(predict_data, target_data, topk=3):
    TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
    for index, words in enumerate(predict_data):
        y_pred, y_true = None, target_data[index]
        if type(predict_data) == str:
            words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
            y_pred = [i[0] for i in words]
        elif type(predict_data) == list:
            y_pred = words
        y_pred = y_pred[0: topk]
        TRUE_NUM = len(set(y_pred) & set(y_true))
        TRUE_COUNT += TRUE_NUM
        PRED_COUNT += len(y_pred)
        GOLD_COUNT += len(y_true)
    if PRED_COUNT != 0:
        p = (TRUE_COUNT / PRED_COUNT)
    else:
        p = 0
    if GOLD_COUNT != 0:
        r = (TRUE_COUNT / GOLD_COUNT)
    else:
        r = 0
    if (r + p) != 0:
        f1 = ((2 * r * p) / (r + p))
    else:
        f1 = 0
    p = round(p * 100, 2)
    r = round(r * 100, 2)
    f1 = round(f1 * 100, 2)
    return p, r, f1

# 假设已经定义了数据集和数据加载器
# train_dataloader = ...
# test_dataloader = ...

# 训练和评估
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertHANModel(num_labels=6)
model = model.to(device)
optim = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

epochs = 5
best_f1 = 0.0
scaler = GradScaler()

for epoch in range(epochs):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i, batch in enumerate(train_dataloader):
        optim.zero_grad()
#input_ids, attention_masks, _, features, tags = batch
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        features = batch[3].to(device)
        tags = batch[4].to(device)

        with autocast():
            pred_tags = model(input_ids, attention_masks,features)

            # 展平 pred_tags 和 tags 以匹配形状
            pred_tags = pred_tags.reshape(-1, pred_tags.size(-1))
            tags = tags.reshape(-1)

            #print(f"pred_tags shape: {pred_tags.shape}, tags shape: {tags.shape}")

            loss = loss_fn(pred_tags, tags)
            loss = loss.mean()

        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        pred_tags = F.softmax(pred_tags, dim=-1)
        pred_tags = torch.argmax(pred_tags, dim=-1)
        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)
        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [], []
    for i, batch in enumerate(test_dataloader):
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        tokens = batch[2]  # tokens 不是 Tensor，直接使用
        features = batch[3].to(device)
        tags = batch[4].to(device)

        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
                    module.train(False)
            with autocast():
                pred_tags = model(input_ids, attention_masks,features)
                pred_tags = F.softmax(pred_tags, dim=-1)
                pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        poss = []
        for i in range(len(tags)):
            pos = []
            for j in range(len(tags[i])):
                if tags[i][j] == 0:
                    pos.append(j)
            poss.append(pos)

        kw_true.extend(TagConvert(tags, tokens))
        kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(), './pretrain_pt/bert_HAtten_EGG.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch + 1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))

    torch.cuda.empty_cache()


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

epoch1:  loss:0.28   train_f1_value:0.16  test_f1_value:0.19  kw_f1_value:0.25
epoch2:  loss:0.24   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.00
epoch3:  loss:0.23   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.37
epoch4:  loss:0.22   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.25
epoch5:  loss:0.21   train_f1_value:0.19  test_f1_value:0.19  kw_f1_value:0.37


In [ ]:
def load_model(model_path, num_labels):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BertHANModel(num_labels=num_labels)
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    return model

def calculate_f1(y_pred, y_true):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    mask = np.where(y_true != 0)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    return y_pred, y_true

def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
        kw_list = []
        nkw_list = ""
        for j in range(len(raw_tags[i])):
            item = raw_tags[i][j]
            if item == 0:
                continue
            if poss != None and j in poss[i]:
                continue
            if item == 4:
                kw_list.append(str(words_set[j][i]))
            if item == 1:
                nkw_list += str(words_set[j][i])
            if item == 2:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
            if item == 3:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
                kw_list.append(nkw_list)
                nkw_list = ""
        true_tags.append(kw_list)
    return true_tags

def evaluate(predict_data, target_data, topk=3):
    TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
    for index, words in enumerate(predict_data):
        y_pred, y_true = None, target_data[index]
        if type(predict_data) == str:
            words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
            y_pred = [i[0] for i in words]
        elif type(predict_data) == list:
            y_pred = words
        y_pred = y_pred[0: topk]
        TRUE_NUM = len(set(y_pred) & set(y_true))
        TRUE_COUNT += TRUE_NUM
        PRED_COUNT += len(y_pred)
        GOLD_COUNT += len(y_true)
    if PRED_COUNT != 0:
        p = (TRUE_COUNT / PRED_COUNT)
    else:
        p = 0
    if GOLD_COUNT != 0:
        r = (TRUE_COUNT / GOLD_COUNT)
    else:
        r = 0
    if (r + p) != 0:
        f1 = ((2 * r * p) / (r + p))
    else:
        f1 = 0
    p = round(p * 100, 2)
    r = round(r * 100, 2)
    f1 = round(f1 * 100, 2)
    return p, r, f1

def predict_and_evaluate(model, dataloader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    label_true, label_pred = [], []
    kw_true, kw_pred = [], []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:

            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            tokens = batch[2]  # tokens 不是 Tensor，直接使用
            features = batch[3].to(device)
            tags = batch[4].to(device)

            pred_tags = model(input_ids, attention_masks,features)
            pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)
            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)
    return label_f1, P, R, F1

# 加载模型
model_path = './pretrain_pt/bert_HAtten_EGG.pt'
num_labels = 6
model = load_model(model_path, num_labels)

# 假设 test_dataloader 已经定义好
label_f1, P, R, F1 = predict_and_evaluate(model, test_dataloader)

print(f"label_f1: {label_f1:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1: {F1:.2f}")

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

label_f1: 0.19, Precision: 0.18, Recall: 37.50, F1: 0.37


In [ ]:
####################层注意力+不同EEG组合

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import BertModel
from torch import nn
from torch.cuda.amp import autocast, GradScaler
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import BertModel
from torch import nn
from torch.cuda.amp import autocast, GradScaler

# 定义 Attention 机制
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.context = nn.Parameter(torch.FloatTensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.context)

    def forward(self, x, mask=None):
        attention_in = torch.tanh(torch.matmul(x, self.context))
        attention_in = torch.squeeze(attention_in, -1)
        if mask is not None:
            attention_in = attention_in * mask.float()
        attention_weights = F.softmax(attention_in, dim=-1)
        weighted_sum = torch.bmm(attention_weights.unsqueeze(1), x).squeeze(1)
        return weighted_sum

# 定义 BertHANModel
class BertHANModel(nn.Module):
    def __init__(self, num_labels, hidden_dim=768, rnn_dim=256, eeg_dim=8):
        super(BertHANModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.word_attention = Attention(hidden_dim)
        self.rnn = nn.GRU(hidden_dim, rnn_dim, batch_first=True, bidirectional=True)
        self.sentence_attention = Attention(rnn_dim * 2)
        self.classifier = nn.Linear(rnn_dim * 2 + eeg_dim, num_labels)

    def forward(self, input_ids, attention_mask, extra_features):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs[0]  # shape: (batch_size, seq_length, hidden_dim)

        # Word-level attention
        word_attention_output = self.word_attention(sequence_output)

        # Sentence-level GRU
        rnn_output, _ = self.rnn(word_attention_output.unsqueeze(1))

        # Sentence-level attention
        sentence_attention_output = self.sentence_attention(rnn_output)

        # 确保 sentence_attention_output 是三维张量
        if sentence_attention_output.dim() == 2:
            sentence_attention_output = sentence_attention_output.unsqueeze(1).expand(-1, extra_features.size(1), -1)

        # 拼接特征
        combined_output = torch.cat((sentence_attention_output, extra_features), dim=-1)  # [batch_size, seq_len, rnn_dim * 2 + eeg_dim]

        logits = self.classifier(combined_output)  # shape: (batch_size, seq_len, num_labels)

        return logits  # 返回 shape: (batch_size, seq_length, num_labels)

# 根据 EEG 组合标签获取相应的维度索引
def get_eeg_dim(combo):
    mappings = {
        'EEG1': (0, 1),
        'EEG2': (1, 2),
        'EEG3': (2, 3),
        'EEG4': (3, 4),
        'EEG5': (4, 5),
        'EEG6': (5, 6),
        'EEG7': (6, 7),
        'EEG8': (7, 8),
        'EEG1.2': (0, 2),
        'EEG2.3': (1, 3),
        'EEG3.4': (2, 4),
        'EEG4.5': (3, 5),
        'EEG5.6': (4, 6),
        'EEG6.7': (5, 7),
        'EEG7.8': (6, 8),
    }
    return mappings[combo]

# 训练和评估函数
def train_and_evaluate(eeg_combo):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    eeg_start, eeg_end = get_eeg_dim(eeg_combo)
    eeg_dim = eeg_end - eeg_start

    model = BertHANModel(num_labels=6, eeg_dim=eeg_dim)
    model = model.to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    loss_fn = nn.CrossEntropyLoss(reduction='none', ignore_index=0)
    loss_fn = loss_fn.to(device)

    epochs = 5
    best_f1 = 0.0
    scaler = GradScaler()

    for epoch in range(epochs):
        loss_value = 0.0
        model.train()
        label_true, label_pred = [], []
        for i, batch in enumerate(train_dataloader):
            optim.zero_grad()
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            features = batch[3][:, :, eeg_start:eeg_end].to(device)
            tags = batch[4].to(device)

            with autocast():
                pred_tags = model(input_ids, attention_masks, features)

                # 展平 pred_tags 和 tags 以匹配形状
                pred_tags = pred_tags.reshape(-1, pred_tags.size(-1))
                tags = tags.reshape(-1)

                loss = loss_fn(pred_tags, tags)
                loss = loss.mean()

            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()

            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)
            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)
            loss_value += loss.item()

        label_train_f1 = f1_score(label_true, label_pred, average='macro')

        model.eval()
        kw_true, kw_pred = [], []
        label_true, label_pred = [], []
        for i, batch in enumerate(test_dataloader):
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            tokens = batch[2]  # tokens 不是 Tensor，直接使用
            features = batch[3][:, :, eeg_start:eeg_end].to(device)
            tags = batch[4].to(device)

            with torch.no_grad():
                for module in model.modules():
                    if isinstance(module, nn.Dropout):
                        module.p = 0
                        module.train(False)
                with autocast():
                    pred_tags = model(input_ids, attention_masks, features)
                    pred_tags = F.softmax(pred_tags, dim=-1)
                    pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)

            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

        label_f1 = f1_score(label_true, label_pred, average='macro')
        P, R, F1 = evaluate(kw_true, kw_pred)

        if F1 > best_f1:
            best_f1 = F1
            torch.save(model.state_dict(), f'/content/drive/MyDrive/ner/pretrain_pt/bert_HAtten_{eeg_combo}.pt')

        print(f"epoch {epoch + 1}: loss: {loss_value / len(train_dataloader):.2f} train_f1: {label_train_f1:.2f} test_f1: {label_f1:.2f} kw_f1: {F1:.2f}")

        torch.cuda.empty_cache()

    return P, R, F1

# 使用不同的 EEG 组合进行训练和评估
eeg_combos = ['EEG1', 'EEG2', 'EEG3', 'EEG4', 'EEG5', 'EEG6', 'EEG7', 'EEG8',
              'EEG1.2', 'EEG2.3', 'EEG3.4', 'EEG4.5', 'EEG5.6', 'EEG6.7', 'EEG7.8']

for eeg_combo in eeg_combos:
    print(f"Training with EEG combo: {eeg_combo}")
    P, R, F1 = train_and_evaluate(eeg_combo)
    print(f"EEG combo: {eeg_combo} - Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")


Training with EEG combo: EEG1
epoch 1: loss: 0.27 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00
epoch 2: loss: 0.24 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00
epoch 3: loss: 0.23 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.24
epoch 4: loss: 0.22 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.24
epoch 5: loss: 0.22 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.24
EEG combo: EEG1 - Precision: 0.12, Recall: 28.57, F1 Score: 0.24
Training with EEG combo: EEG2
epoch 1: loss: 0.27 train_f1: 0.16 test_f1: 0.19 kw_f1: 0.00
epoch 2: loss: 0.24 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.00
epoch 3: loss: 0.23 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.57
epoch 4: loss: 0.22 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.49
epoch 5: loss: 0.22 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.24
EEG combo: EEG2 - Precision: 0.12, Recall: 28.57, F1 Score: 0.24
Training with EEG combo: EEG3
epoch 1: loss: 0.28 train_f1: 0.16 test_f1: 0.19 kw_f1: 0.12
epoch 2: loss: 0.24 train_f1: 0.19 test_f1: 0.19 kw_f1: 0.24
epoch 3: loss: 0.23 train_f1: 0.19 test_f1: 0.19